# Introduction to satellite image classification with Python
## Unsupervised classification
Stephane Goldstein [(LinkedIn)](https://www.linkedin.com/in/stephane-goldstein/)

Geogeeks Perth 2023-05-10

## Toolset

### [rasterio](https://rasterio.readthedocs.io/en/stable/intro.html)

![figure_01](./figures/figure_01.png)

### [scikit-learn](https://scikit-learn.org)

![figure_03](./figures/figure_03.png)

![figure_04](./figures/figure_04.png)

## Example image
- 15 x 15 pixels 20m resolution Sentinel-2 image subset

![figure_06](./figures/figure_06.png)

![figure_07](./figures/figure_07.png) ![figure_08](./figures/figure_08.png)

![figure_05](./figures/figure_05.png)

In [ ]:
# Create a string with the path of each tif (band) file
# Store in a List

tif_base_path = './L2A_T50HLK_A039668_20230126T022316/bands'
tif_base_name = 'T50HLK_20230126T021341_'

tif_bands = [
    'B02', 'B03', 'B04', 'B05', 'B06', 'B07',
    'B8A', 'B11', 'B12'
    ]

tif_files = [f'{tif_base_path}/{tif_base_name}{b}_20m.tif' for b in tif_bands]

print('\n'.join([t for t in tif_files]))

## Data and Metadata
Geographic raster data is usually composed of
* data: pixel values
* metadata: CRS, location, resolution, data type, No Data piexel value, etc

The metadata is the same for all bands, so only the metadata of the first file will be stored.

Rasterio calls the metadata the `profile` and it is basically a Python dictionary:

In [ ]:
# Import rasterio
import rasterio

# Get first file metadata
with rasterio.open(tif_files[0]) as rst:
    bands_profile = rst.profile

# Print metadata
for k, v in bands_profile.items():
    print(f'{k}:{v}\n')

* [Affine GitHub reposirory](https://github.com/rasterio/affine)

* Short article on the Affine transform from [perrygeo](https://www.perrygeo.com/python-affine-transforms.htm)

## Data
Rasterio reads raster data into a NumPy array


In [ ]:
# Intialize an empty list to store the data
bands_data = []

# Iterate over the tif files
for tif in tif_files:
    with rasterio.open(tif) as rst:
        # Read the first band of the tif file into a NumPy array object
        data = rst.read(1)
        # Append the NumPy array into the list
        bands_data.append(data)


The data is at this stage a list of NumPy arrays:


In [ ]:
print(bands_data[0])

In [ ]:
print(bands_data[0].shape)

The list needs to be converted to a single a NumPy array

In [ ]:
import numpy as np
bands_data = np.array(bands_data)

# Print shape of the NumPy array
print(bands_data.shape)


In [ ]:
# Extract each dimension into a descriptive variable
bands_count = bands_data.shape[0] # number of bands
height = bands_data.shape[1]      # number of rows = Y
width = bands_data.shape[2]       # number of colums = X

---

### :warning: **Warning**

Geographical coordinates are usually (x,y), while NumPY arrays are indexed by (row,col)

x = col

y = row

---

### What does the array look like

The data looks now somewhat like this:

```
image_data = np.array(
    [
        [
            [10,11,12],
            [20,21,22],
            [30,31,32]
        ],
        [
            [40,41,52],
            [50,51,52],
            [60,61,62]
        ],
        [
            [70,71,72],
            [80,81,82],
            [90,91,92]
        ]
    ]

image_data.shape = (3,3,3)
data = rst.read(1)
# 3 bands
# 3 rows
# 3 columns
```

A multiband raster is a 3D array.

scikit-learn works with 2 dimensional data in the format `(features, attributes)`, so something like:

```
data = np.array(
    [
        [10, 40, 70],
        [11, 41, 71],
        [12, 52, 72],
        [20, 50, 80],
        [21, 51, 81],
        [22, 52, 82],
        [30, 60, 90],
        [31, 61, 91],
        [32, 62, 92]
    ]
)
``` 


In the case of the test image, it corresponds to an array of shape (15 * 15, 9) = (225,9)

i.e.:
```
[
    [pixel-1_band-1, pixel-1_band-2, ..., pixel-1_band_9],
    [pixel-2_band-1, pixel-2_band-2, ..., pixel-2_band_9],
    [...],
    [pixel-224_band-1, pixel-224_band-2, ..., pixel-224_band_9],
    [pixel-225_band-1, pixel-225_band-2, ..., pixel-225_band_9]
]
```

#### Step 1: transform the from 3D to 2D

Do not touch the first dimension (number of bands)

Reshape rows and columns into a single long line of data

i.e. concatenate all rows from the raster

In [ ]:
bands_data_flat = bands_data.reshape(bands_count, height * width)
print(bands_data_flat.shape)


### What we have now is of the shape `(attributes, features)`:

```
bands_data_flat = np.array(
    [
        [10, 11, 12, 20, 21, 22, 30, 31, 32],
        [40, 41, 52, 50, 51, 52, 60, 61, 62],
        [70, 71, 72, 80, 81, 82, 90, 91, 92]
    ]
)
bands_data_flat.shape = (3, 9)
```

Or, in the case of the test image
 
```
bands_data_flat = np.array(
    [
        [band-1_pixel_1, band-1_pixel-2, ..., band-1_pixel-224, band-1_pixel-225],
        [band-2_pixel_1, band-2_pixel-2, ..., band-2_pixel-224, band_2_pixel-225],
        [...],
        [band-8_pixel_1, band-8_pixel-2, ..., band-8_pixel-224, band_8_pixel-225],
        [band-9_pixel_1, band-9_pixel-2, ..., band-9_pixel-224, band_9_pixel-225],
    ]
)

bands_data_flat.shape = (9, 225)
```

### Step 2: transpose

In [ ]:
bands_data_flat = bands_data_flat.transpose()
print(bands_data_flat.shape)

## Unsupervised classification

In [ ]:
from sklearn.cluster import KMeans

# ! class is reserved word in Python
n_classes = 6

# Initialize K-means object with desired parameters
kmeans = KMeans(n_clusters=n_classes)
# Run classification
kmeans.fit(bands_data_flat)

# Classification results are stored in the labels_ attribute
kmeans_data_flat = kmeans.labels_

print(kmeans_data_flat.shape)
print('\n')
print(kmeans_data_flat)

The shape of the results is a 1D array of length 'number of pixels' (features)

### Reshape the array back to 2D having the size of the original image

In [ ]:
kmeans_data = kmeans_data_flat.reshape(height, width)
print(kmeans_data)

### Get some statistics on the results

`np.unique` lists unique value and optionl information (e.g. count) on each value

In [ ]:
# Get unique values and count occurences
classes_count = np.unique(kmeans_data, return_counts=True)

print(classes_count)

### Calculate area of each class:

In [ ]:
pixel_size = 20

for i,j in zip(classes_count[0], classes_count[1]):
    class_area = j * pixel_size**2 / 10000
    print(f'Class {i}: {j} ha')

## Export result to tif file

### Change data type

The data is almost ready. But the current data type is int32.

We don't need to use that much bit depth / disk space.

8 bits is enough to store 6 classes

In [ ]:
print(kmeans_data.dtype)
kmeans_data_int8 = kmeans_data.astype('int8')
print(kmeans_data_int8.dtype)

### Prepare metadata

In [ ]:
# Make a copy of the first band profile
kmeans_profile = bands_profile.copy()
print(kmeans_profile['dtype'])
kmeans_profile['dtype'] = 'int8'
print(kmeans_profile['dtype'])


### Write to file

In [ ]:
kmeans_tif_path = './L2A_T50HLK_A039668_20230126T022316/kmeans.tif'

with rasterio.open(kmeans_tif_path, 'w', **kmeans_profile) as rst:
    rst.write(kmeans_data_int8, 1)

### The same process can be used for other scikit-learn [clustering algorithms](https://scikit-learn.org/stable/modules/clustering.html)

(maybe not all of them)

![figure_09](./figures/figure_09.png)

#### Repeat with Agglomerative Clustering:

In [ ]:
from sklearn.cluster import AgglomerativeClustering
agg_clustering = AgglomerativeClustering(n_clusters=6)
agg_clustering.fit(bands_data_flat)
agg_clustering_data_flat = agg_clustering.labels_
agg_clustering_data = agg_clustering_data_flat.reshape(height, width)
agg_clustering_data_int8 = agg_clustering_data.astype('int8')
agg_clustering_profile = bands_profile.copy()
agg_clustering_profile['dtype'] = 'int8'
agg_clustering_tif_path = './L2A_T50HLK_A039668_20230126T022316/agg_clustering.tif'
with rasterio.open(agg_clustering_tif_path, 'w', **agg_clustering_profile) as rst:
    rst.write(agg_clustering_data_int8,1)